###  Imports

In [1]:
import torch
from model import create_model
from utils import FeatureExtractor

import config

import fiftyone as fo
import fiftyone.zoo as foz
from fiftyone import ViewField as F

from dataset import FiftyOneTorchDataset, get_transforms
from model import create_model
from utils import add_detections

from engine import train_model
import config

torch.manual_seed(1)

In [2]:
print(fo.list_datasets())

[]


In [3]:
if len(fo.list_datasets()) > 0:
    dataset = fo.load_dataset("coco-2017-validation")
    dataset.delete()

In [8]:
#Load in the dataset from the FiftyOne model Zoo
fo_dataset = foz.load_zoo_dataset("coco-2017", "validation")

#needed to calculate image height and width
fo_dataset.compute_metadata()

session = fo.launch_app(fo_dataset)

subset = True

train_transforms, test_transforms = get_transforms()

if subset:
    # to filter certain items from the dataset we can
    item_list = ["car", "dog", "bus", 'fork', 'tie', 'person', 'airplane']
    #item_list = ['airplane']
    item_view = fo_dataset.filter_labels("ground_truth",
            F("label").is_in(item_list))

    #session.view = item_view

    # split the dataset in train and test set
    train_view = item_view.take((len(item_view) * config.TRAIN_TEST_SPLIT), seed=51)
    test_view = item_view.exclude([s.id for s in train_view])

else:
    train_view = fo_dataset.take(len(fo_dataset) * config.TRAIN_TEST_SPLIT)
    test_view = fo_dataset.exclude([s.id for s in train_view])
    item_list = fo_dataset.distinct("ground_truth.detections.label")

print(f'Traning on {len(train_view)} samples')
print(f'Testing on {len(test_view)} samples')

# use our dataset and defined transformations
train_dataset = FiftyOneTorchDataset(train_view, train_transforms,
        classes=item_list)
evaluation_dataset = FiftyOneTorchDataset(test_view, test_transforms,
        classes=item_list)

#this is needed for later use, but not for creating the dataset
if item_list[0] != 'background':
     item_list.insert(0,'background')

Found annotations at 'C:\Users\blain\fiftyone\coco-2017\raw\instances_val2017.json'
Images already downloaded
Existing download of split 'validation' is sufficient
Loading existing dataset 'coco-2017-validation'. To reload from disk, either delete the existing dataset or provide a custom `dataset_name` to use


Traning on 2505 samples
Testing on 627 samples


# Check Faster RCNN performance

In [11]:
MODEL_TYPE = 'CLIP-Backbone-FRCNN'
CHECKPOINT_NAME = f'{MODEL_TYPE}_epoch_59.pth'

if item_list[0] != 'background':
     item_list.insert(0,'background')

frcnn_model = create_model(MODEL_TYPE, classes=item_list)
checkpoint = torch.load(CHECKPOINT_NAME)
frcnn_model = create_model(MODEL_TYPE, classes=item_list)

frcnn_model.load_state_dict(checkpoint)
frcnn_model.eval()

add_detections(frcnn_model, evaluation_dataset, fo_dataset, field_name="frcnn_predictions")

results = fo.evaluate_detections(
    test_view,
    "frcnn_predictions",
    classes=item_list,
    eval_key="eval",
    compute_mAP=True
)
session.view = item_view
print(f'mAP: {results.mAP()}')
results.print_report()

Using device cuda
 100% |███████████████████| 20/20 [1.1s elapsed, 0s remaining, 18.8 samples/s]         
Evaluating detections...
 100% |███████████████████| 20/20 [113.4ms elapsed, 0s remaining, 176.4 samples/s] 
Performing IoU sweep...
 100% |███████████████████| 20/20 [99.1ms elapsed, 0s remaining, 201.7 samples/s] 


mAP: 0.5763621393849095
              precision    recall  f1-score   support

  background       0.00      0.00      0.00         0
    airplane       1.00      0.93      0.96        28

   micro avg       1.00      0.93      0.96        28
   macro avg       0.50      0.46      0.48        28
weighted avg       1.00      0.93      0.96        28



# Check CLIP FRCNN performance

In [7]:
# test out the trained CLIP-FRCNN
MODEL_TYPE = 'CLIP-FRCNN'
CHECKPOINT_NAME = f'{MODEL_TYPE}_epoch_105.pth'


# tokenize item list for CLIP
import clip
_, preprocess = clip.load("RN50", device=config.DEVICE)

if item_list[0] != 'background':
     item_list.insert(0,'background')

text_tokens = clip.tokenize(["This is " + desc for desc in item_list]).cuda()


clip_frcnn_model = create_model(MODEL_TYPE, classes=text_tokens)
checkpoint = torch.load(CHECKPOINT_NAME)
clip_frcnn_model = create_model(MODEL_TYPE, classes=text_tokens)

clip_frcnn_model.load_state_dict(checkpoint)
clip_frcnn_model.eval()

add_detections(clip_frcnn_model, evaluation_dataset, fo_dataset, field_name="clip_frcnn_predictions")

results = fo.evaluate_detections(
    test_view,
    "clip_frcnn_predictions",
    classes=item_list,
    eval_key="clip_eval",
    compute_mAP=True
)

print(f'mAP: {results.mAP()}')
results.print_report()

session.view = test_view

Using device cuda
   0% |/----------------|   1/627 [212.4ms elapsed, 2.2m remaining, 4.7 samples/s] 

C:\Users\blain\anaconda3\envs\torch-frcnn\lib\site-packages\torch\functional.py:445: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ..\aten\src\ATen\native\TensorShape.cpp:2157.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


 100% |█████████████████| 627/627 [47.9s elapsed, 0s remaining, 10.7 samples/s]      
Evaluating detections...
 100% |█████████████████| 627/627 [11.8s elapsed, 0s remaining, 53.2 samples/s]      
Performing IoU sweep...
 100% |█████████████████| 627/627 [13.5s elapsed, 0s remaining, 46.5 samples/s]      
mAP: 0.02026706541596025
              precision    recall  f1-score   support

  background       0.00      0.00      0.00         0
         car       0.11      0.26      0.15       459
         dog       0.01      0.39      0.01        41
         bus       0.01      0.22      0.01        74
        fork       0.00      0.02      0.00        44
         tie       0.00      0.03      0.00        29
      person       0.42      0.45      0.43      2375
    airplane       0.00      0.45      0.01        22

   micro avg       0.09      0.40      0.15      3044
   macro avg       0.07      0.23      0.08      3044
weighted avg       0.35      0.40      0.36      3044



In [8]:
print(f'rpn nms thresh: {clip_frcnn_model.rpn.nms_thresh}')
print(f'rpn score thresh: {clip_frcnn_model.rpn.score_thresh}')

rpn nms thresh: 0.7
rpn score thresh: 0.9
